## Calculating Instrument Visibility Windows

API Version: 1.0

Authors: Craig Pellegrino (NASA GSFC)

### Introduction

This notebook will demonstrate how to use the ACROSS `Client` to calculate target visibility windows for both a single instrument and jointly between many instruments. Visibility windows give the possible time ranges when an instrument is potentially able to observe a target without violating any of its constraints. Common constraints which we use include Sun angle constraints (i.e. how close an instrument can look at the sun), Earth limb constraints (i.e. how close an instrument can look at the Earth), and South Atlantic Anomaly constraints (durations where an instrument cannot observe because it is traveling through the South Atlantic Anomaly).

The ACROSS `core-server` tracks these constraints for each instrument in its system and uses them to find visibility windows for a given target and instrument. The `core-server` exposes API endpoints to calculate these visibility windows both for a single instrument and between many instruments at the same time. Here we will demonstrate how to use the `Client` to interface with this API in order to retrieve possible visibility windows.

### 1. Setup

We'll begin by instantiating a `Client` object. The `Client` will allow us to retrieve `Instrument` IDs from the ACROSS `core-server` and request visibility windows for a target to be observed by those instruments. 

Let's test out the visibility calculator functionality using Swift's UVOT. First, let's retrieve the UVOT instrument ID from the server. For more demonstration on how to do this, see the example notebook `001_accessing_SSA_model_data`.

In [1]:
from across.client import Client

client = Client()

uvot = client.instrument.get_many(name="UVOT")[0]
uvot.id

'1c074192-f6d5-465d-844e-85a0010b2d87'

### 2. Calculating Visibility Windows

The next parameters we need for the visibility calculation are a target position and a date range. Let's set the R.A., Declination, and date range begin and end values:

In [2]:
from datetime import datetime

# RA and dec should be floats or ints
target_ra = 83.86662
target_dec = -69.26986

# Date range begin and end should be datetime objects
date_range_begin = datetime(2025, 10, 23, 0, 0, 0)
date_range_end = datetime(2025, 10, 24, 0, 0, 0)

Now we have everything we need for the calculation. Let's do it! We just need to call the `calculate_windows` method of `client.visibility_calculator` using the above inputs: 

In [3]:
visibility_windows = client.visibility_calculator.calculate_windows(
    instrument_id=uvot.id,
    ra=target_ra,
    dec=target_dec,
    date_range_begin=date_range_begin,
    date_range_end=date_range_end,
)
print(visibility_windows.model_dump_json(indent=4))

{
    "instrument_id": "1c074192-f6d5-465d-844e-85a0010b2d87",
    "visibility_windows": [
        {
            "window": {
                "begin": {
                    "datetime": "2025-10-23T00:00:00",
                    "constraint": "Window",
                    "observatory_id": "d5faadca-c0bb-4bb8-b4e4-75149b435ae8"
                },
                "end": {
                    "datetime": "2025-10-23T00:28:00",
                    "constraint": "Earth Limb",
                    "observatory_id": "d5faadca-c0bb-4bb8-b4e4-75149b435ae8"
                }
            },
            "max_visibility_duration": 1679,
            "constraint_reason": {
                "start_reason": "Observatory Window",
                "end_reason": "Observatory Earth Limb"
            }
        },
        {
            "window": {
                "begin": {
                    "datetime": "2025-10-23T01:43:00",
                    "constraint": "South Atlantic Anomaly",
                    "obse

That's it! `visibility_windows` contains a list of `VisibilityWindow` objects. Each `VisibilityWindow` contains a begin and end time, a duration, and a list of the constraints that cause the window to open or close. 

One last note about the `calculate_windows` method: it also contains two optional parameters. The first, `hi_res`, is a boolean flag that calculates the windows in minute intervals if set to `True`, or hour intervals if set to `False` (the default). The second, `min_visibility_duration`, is an integer (that defaults to `0`) which will only return windows with a duration greater than the input value. 

### 3. Using Visibility Windows

Now that we've calculated visibility windows for a target, let's take a closer look at the returned objects and the data they contain. 

Let's examine a `Window` and walk through its attributes, step by step:

In [4]:
window = visibility_windows.visibility_windows[0]

First off, `Window` has `begin` and `end` attributes which are `ConstrainedDates`--they combine a normal `datetime` with a `Constraint`, describing why the window either began or ended:

In [5]:
print(window.window.begin.model_dump_json(indent=4))
print(window.window.end.model_dump_json(indent=4))

{
    "datetime": "2025-10-23T00:00:00",
    "constraint": "Window",
    "observatory_id": "d5faadca-c0bb-4bb8-b4e4-75149b435ae8"
}
{
    "datetime": "2025-10-23T00:28:00",
    "constraint": "Earth Limb",
    "observatory_id": "d5faadca-c0bb-4bb8-b4e4-75149b435ae8"
}


As we can see, the first window begins because we've begun our calculation, and ends when the instrument must point too close to the Earth limb to observe the target. 

`Window` also has a `max_visibility_duration`, giving the duration of the window (in seconds), as well as a `ConstraintReason`, which sums up the constraints leading to the window beginning and ending:

In [6]:
print(f"Duration of the window: {window.max_visibility_duration}")
print(f"Constraint reasons:\n {window.constraint_reason}")

Duration of the window: 1679
Constraint reasons:
 start_reason='Observatory Window' end_reason='Observatory Earth Limb'


As a last example, let's print out the start times, end times, and constraint reasons for the first few windows returned by our calculation:

In [7]:
print("Start time:            End time:            Length:          Reason:   \n")
for window in visibility_windows.visibility_windows[:5]:
    print(
        window.window.begin.datetime,
        " | ",
        window.window.end.datetime,
        " | ",
        window.max_visibility_duration,
        " | ",
        window.constraint_reason,
    )

Start time:            End time:            Length:          Reason:   

2025-10-23 00:00:00  |  2025-10-23 00:28:00  |  1679  |  start_reason='Observatory Window' end_reason='Observatory Earth Limb'
2025-10-23 01:43:00  |  2025-10-23 02:01:00  |  1080  |  start_reason='Observatory South Atlantic Anomaly' end_reason='Observatory Earth Limb'
2025-10-23 03:21:00  |  2025-10-23 03:34:00  |  780  |  start_reason='Observatory South Atlantic Anomaly' end_reason='Observatory Earth Limb'
2025-10-23 04:31:00  |  2025-10-23 04:32:00  |  59  |  start_reason='Observatory Earth Limb' end_reason='Observatory South Atlantic Anomaly'
2025-10-23 04:59:00  |  2025-10-23 05:07:00  |  479  |  start_reason='Observatory South Atlantic Anomaly' end_reason='Observatory Earth Limb'


Finally, we can compare these values to those calculated by the Swift team's own visibility calculator [here](https://www.swift.psu.edu/operations/visibility.php). Plugging in the same coordinates and date range gives the following:

```
Begin	End	Length
2025-10-23 00:00:00	2025-10-23 00:28:00	1680 seconds
2025-10-23 01:43:00	2025-10-23 02:01:00	1080 seconds
2025-10-23 03:22:00	2025-10-23 03:34:00	720 seconds
2025-10-23 04:31:00	2025-10-23 04:32:00	60 seconds
2025-10-23 04:59:00	2025-10-23 05:07:00	480 seconds
```

in agreement with our calculations!

### 4. Calculating and Using Joint Visibility Windows

The ACROSS `Client` takes this functionality one step further by enabling the calculation of joint visibility windows between many different instruments. This is useful if a user wants to schedule simultaneous observations of a target on multiple instruments, which is often necessary to achieve complementary wavelength or temporal coverage. This process is very simple in the ACROSS system and utilizes the same procedures discussed above.

Let's walk through an example calculation. Let's say you want to know when both Swift UVOT and the ACIS High Energy Transmission Grating on Chandra can observe the target simultaneously. First, we need the ACROSS IDs of the instruments for which we wish to calculate the joint visibility: 

In [8]:
# Retrieve both the UVOT and ACIS UUIDs

uvot_id = client.instrument.get_many(name="UVOT")[0].id
acis_id = client.instrument.get_many(name="ACIS-HETG")[0].id

Now, we simply pass those instrument IDs into the `client.visibility_calculate.calculate_joint_windows()` method, which takes the same parameters as the single instrument `calculate_windows()` method above. For simplicity, let's use the same target coordinate and date range as in the previous example, and calculate the joint visibility windows of the target on both Swift UVOT and ACIS:

In [9]:
joint_visibility_windows = client.visibility_calculator.calculate_joint_windows(
    instrument_ids=[uvot.id, acis_id],
    ra=target_ra,
    dec=target_dec,
    date_range_begin=date_range_begin,
    date_range_end=date_range_end,
)
print(joint_visibility_windows.model_dump_json(indent=4))

{
    "instrument_ids": [
        "1c074192-f6d5-465d-844e-85a0010b2d87",
        "e570ae2a-16a5-4a75-b7f2-7905e4b4357c"
    ],
    "visibility_windows": [
        {
            "window": {
                "begin": {
                    "datetime": "2025-10-23T00:00:00",
                    "constraint": "Window",
                    "observatory_id": "1c074192-f6d5-465d-844e-85a0010b2d87"
                },
                "end": {
                    "datetime": "2025-10-23T00:27:59.999900",
                    "constraint": "Earth Limb",
                    "observatory_id": "1c074192-f6d5-465d-844e-85a0010b2d87"
                }
            },
            "max_visibility_duration": 1679,
            "constraint_reason": {
                "start_reason": "Observatory Window",
                "end_reason": "Observatory Earth Limb"
            }
        },
        {
            "window": {
                "begin": {
                    "datetime": "2025-10-23T01:42:59.999632",
      

The calculation returns three data components: `instrument_ids`, i.e. the IDs of the instruments used in the calculation; `visibility_windows`, the windows of joint visibility; and `observatory_visibility_windows`, which give the individual windows for each of the instruments. Each of `visibility_windows` and `observatory_visibility_windows` have the same `VisibilityWindow` objects as described in the previous example, and can be used the same way. Looking at the data, one can see that most of the windows are driven by Swift's constraints, although there is a roughly two hour period where Chandra is blocked by the Earth limb but the target remains visible to Swift UVOT for part of that time. The power of the ACROSS system lies in performing the calculations for each individual instrument, finding the windows of overlap between all instruments, and providing all this information back to the user so that it may be used to inform scheduling and observation requests.